In [ ]:
import ee
import os
from pathlib import Path
from datetime import datetime as dt
from abf import abf

In [ ]:
__version__ = ""
exec(Path(f"{os.getcwd()}/src/abf/version.py").read_text(encoding="utf-8"))

In [ ]:
# Initialize Google Earth Engine
# ee.Authenticate() (might be needed in some situations)
ee.Initialize()

In [ ]:
# Data path
folderRoot = os.getcwd() + "/results"
if not os.path.exists(folderRoot):
    os.mkdir(folderRoot)

# Images path
folderCache = os.getcwd() + "/cache"
if not os.path.exists(folderCache):
    os.mkdir(folderCache)
    
# Selected image date to dectection
selectedDate = "2019-06-03"

# folder to save results from algorithm at
folder = (
    folderRoot
    + "/"
    + dt.now().strftime("%Y%m%d_%H%M%S")
    + f"[v={str(__version__)}-sandbox,d={selectedDate},t=180,m=ocsvm,s=modis]")
if not os.path.exists(folder):
    os.mkdir(folder)

In [ ]:
# fix permissions
!chmod 777 $folderRoot
!chmod 777 $folder
!chmod 777 $folderCache

In [ ]:
# Latitude and Longitude (PoI)
lat_lon = "-83.48811946836814,41.85776095627803,-83.18290554014548,41.677617395337826"

# (colon, lat and lon separated by comma, all together)
# and dates by the user (two dates, beginning and end, separated by commas)
x1, y1, x2, y2 = lat_lon.split(",")

# Assemble Geometry on Google Earth Engine
geometry = ee.Geometry.Polygon(
    [
        [
            [float(x1), float(y2)],
            [float(x2), float(y2)],
            [float(x2), float(y1)],
            [float(x1), float(y1)],
            [float(x1), float(y2)],
        ]
    ]
)

In [ ]:
# create algorithm object
algorithm = abf.Abf(days_threshold=180,
                    grid_size=12,
                    sensor="modis", 
                    geometry=geometry,
                    lat_lon=lat_lon,
                    cache_path=folder,
                    force_cache=False,
                    days_in=5,
                    days_out=5,
                    from_date="2019-07-11",
                    model="rf",
                    reducer=True,
                    class_mode=True,
                    attribute_lat_lon=False)

In [ ]:
# preprocessing
algorithm.process_timeseries_data()
algorithm.process_training_data(df=algorithm.df_timeseries)

In [ ]:
# train/predict
algorithm.train(batch_size=2048, disable_gpu=True)
algorithm.predict(folder=folder+"/prediction")

In [ ]:
# prediction results
algorithm.save_dataset(df=algorithm.df_results, path=folder+'/results.csv')

In [ ]:
# preprocessing results
algorithm.save_dataset(df=algorithm.df_timeseries, path=folder+'/timeseries.csv')